In [1]:
from pydna.amplify import Anneal
from pydna.amplify import pcr
from pydna.assembly import Assembly
from pydna.genbank import genbank
from pydna.genbank import Genbank
from pydna.download import download_text
from pydna.dseqrecord import Dseqrecord
from pydna.dseq import Dseq
from pydna.readers import read
from pydna.readers import read_primer
from pydna.parsers import parse
from pydna.parsers import parse_primers
from pydna.editor import ape
from pydna.design import primer_design
from pydna.design import assembly_fragments
from pydna.design import circular_assembly_fragments
from pydna.utils import eq
from pydna.genbankfixer import gbtext_clean
from pydna.myprimers import PrimerList

In [47]:
import sys
from joblib import Memory
memory = Memory(location=f"{sys.path[0]}/joblib_memory_cachedir", verbose=0)

In [48]:
from pydna.amplify import pcr
from pydna.parsers import parse
from pydna.design import primer_design

from pydna.genbankrecord import GenbankRecord
from IPython.display import display_markdown

In [49]:
import os
from pathlib import Path

In [50]:
sequence_library_path = Path("sequences")

In [51]:
orfs = parse("""\

>ATF1 uSEGUID r8BaXu2MdYByHuhSYXBNKXIxIfg
ATGAATGAAATCGATGAGAAAAATCAGGCCCCCGTGCAACAAGAATGCCTGAAAGAGATGATTCAGAATGGGCATGCTCGGCGTATGGGATCTGTTGAAGATCTGTATGTTGCTCTCAACAGACAAAACTTATATCGAAACTTCTGCACATATGGAGAATTGAGTGATTACTGTACTAGGGATCAGCTCACATTAGCTTTGAGGGAAATCTGCCTGAAAAATCCAACTCTTTTACATATTGTTCTACCAACAAGATGGCCAAATCATGAAAATTATTATCGCAGTTCCGAATACTATTCACGGCCACATCCAGTGCATGATTATATTTCAGTATTACAAGAATTGAAACTGAGTGGTGTGGTTCTCAATGAACAACCTGAGTACAGTGCAGTAATGAAGCAAATATTAGAAGAATTCAAAAATAGTAAGGGTTCCTATACTGCAAAAATTTTTAAACTTACTACCACTTTGACTATTCCTTACTTTGGACCAACAGGACCGAGTTGGCGGCTAATTTGTCTTCCAGAAGAGCACACAGAAAAGTGGAAAAAATTTATCTTTGTATCTAATCATTGCATGTCTGATGGTCGGTCTTCGATCCACTTTTTTCATGATTTAAGAGACGAATTAAATAATATTAAAACTCCACCAAAAAAATTAGATTACATTTTCAAGTACGAGGAGGATTACCAATTATTGAGGAAACTTCCAGAACCGATCGAAAAGGTGATAGACTTTAGACCACCGTACTTGTTTATTCCGAAGTCACTTCTTTCGGGTTTCATCTACAATCATTTGAGATTTTCTTCAAAAGGTGTCTGTATGAGAATGGATGATGTGGAAAAAACCGATGATGTTGTCACCGAGATCATCAATATTTCACCAACAGAATTTCAAGCGATTAAAGCAAATATTAAATCAAATATCCAAGGTAAGTGTACTATCACTCCGTTTTTACATGTTTGTTGGTTTGTATCTCTTCATAAATGGGGTAAATTTTTCAAACCATTGAACTTCGAATGGCTTACGGATATTTTTATCCCCGCAGATTGCCGCTCACAACTACCAGATGATGATGAAATGAGACAGATGTACAGATATGGCGCTAACGTTGGATTTATTGACTTCACCCCCTGGATAAGCGAATTTGACATGAATGATAACAAAGAAAATTTTTGGCCACTTATTGAGCACTACCATGAAGTAATTTCGGAAGCTTTAAGAAATAAAAAGCATCTCCATGGCTTAGGGTTCAATATACAAGGCTTCGTTCAAAAATATGTGAACATTGACAAGGTAATGTGCGATCGTGCCATCGGGAAAAGACGCGGAGGTACATTGTTAAGCAATGTAGGTCTGTTTAATCAGTTAGAGGAGCCCGATGCCAAATATTCTATATGCGATTTGGCATTTGGCCAATTTCAAGGATCCTGGCACCAAGCATTTTCCTTGGGTGTTTGTTCGACTAATGTAAAGGGGATGAATATTGTTGTTGCTTCAACAAAGAATGTTGTTGGTAGTCAAGAATCTCTCGAAGAGCTTTGCTCCATTTACAAAGCTCTCCTTTTAGGCCCTTAG

>CTT1 uSEGUID unRT3TWPEXorUhgOrGZRvwcsZwU
ATGAACGTGTTCGGTAAAAAAGAAGAAAAGCAAGAAAAAGTTTACTCTCTACAAAACGGTTTTCCGTACTCTCATCACCCATACGCTTCTCAATACTCAAGACCAGACGGCCCTATCTTACTGCAAGACTTCCATCTGCTGGAAAATATCGCAAGTTTCGATAGAGAAAGAGTTCCGGAGCGTGTAGTCCATGCCAAAGGTGGTGGTTGTAGACTGGAGTTCGAACTAACAGATTCTTTGAGTGATATTACATACGCCGCTCCATACCAGAATGTGGGTTACAAATGTCCTGGTCTTGTTCGTTTTTCCACCGTTGGTGGTGAAAGTGGTACACCAGACACTGCAAGAGACCCAAGAGGTGTTTCTTTTAAATTCTATACCGAGTGGGGGAACCATGACTGGGTCTTCAACAATACTCCCGTCTTCTTCCTCAGAGACGCTATTAAGTTTCCCGTATTTATTCATTCGCAAAAGAGAGACCCTCAGTCTCATCTGAATCAGTTTCAGGACACTACCATATACTGGGATTATCTAACATTGAATCCGGAATCAATCCATCAAATAACTTACATGTTTGGTGATAGAGGTACTCCTGCTTCGTGGGCTAGTATGAACGCGTACTCTGGTCATTCCTTCATCATGGTCAACAAAGAAGGTAAGGACACATATGTGCAATTCCACGTCTTGTCGGATACTGGTTTTGAAACCTTGACTGGAGATAAGGCTGCTGAACTGTCAGGCTCCCACCCTGATTATAATCAGGCAAAGCTGTTCACTCAATTGCAAAATGGCGAAAAGCCAAAATTTAACTGTTATGTGCAAACAATGACACCCGAACAAGCAACTAAGTTCAGGTATTCGGTAAATGACCTAACGAAAATATGGCCACACAAGGAATTCCCTTTGAGAAAATTTGGTACCATCACCCTAACGGAGAATGTTGACAATTATTTCCAAGAAATTGAACAAGTTGCATTCAGTCCAACGAACACTTGTATCCCAGGTATTAAGCCTTCTAATGATTCCGTTCTACAAGCCAGACTTTTCTCCTATCCAGACACTCAACGTCATAGATTGGGAGCCAACTATCAGCAATTGCCCGTCAACAGACCAAGAAACTTGGGATGTCCATACTCCAAAGGTGATTCCCAATACACTGCCGAACAGTGTCCATTTAAAGCAGTGAACTTCCAAAGGGACGGCCCAATGAGTTACTACAATTTCGGTCCTGAGCCAAATTATATTTCCAGTTTACCAAATCAAACTCTGAAATTCAAAAATGAAGACAACGACGAAGTATCTGATAAGTTCAAAGGGATAGTTCTTGACGAAGTAACAGAAGTTTCTGTGAGAAAACAGGAACAAGACCAAATCAGAAACGAGCATATTGTTGATGCCAAAATTAATCAATATTACTACGTTTATGGTATTAGTCCACTAGACTTCGAACAGCCAAGAGCTCTATATGAAAAGGTATACAACGATGAACAGAAGAAATTATTCGTTCATAACGTTGTTTGCCACGCTTGTAAGATCAAAGATCCTAAAGTCAAAAAGAGAGTTACGCAATACTTTGGTTTGCTAAACGAAGATTTGGGTAAAGTCATTGCAGAAGGCTTGGGAGTTCCTTGGGAACCTGTTGACCTTGAAGGTTATGCCAAGACTTGGTCCATTGCAAGTGCCAATTAA

""")

In [54]:
@memory.cache
def genbank_accession(s:str):
    
    from Bio.Blast import NCBIWWW
    from Bio.Blast import NCBIXML

    result_handle = NCBIWWW.qblast("blastn",
                                   "nt",
                                    s,
                                    hitlist_size=10,
                                    alignments = 1,
                                    expect=1e-8)

    blast_records = NCBIXML.read(result_handle)

    results = []
    length = len(s)

    for alignment in blast_records.alignments:
        
        for hsp in alignment.hsps:
            
            if hsp.identities == length: # only **perfect** matches !
                
                results.append((alignment.accession, *sorted((hsp.sbjct_start, hsp.sbjct_end))))

    return results

In [59]:
genes = []
for orf in orfs:
    display_markdown(f"# {orf.id}", raw=True)
    assert orf.useguid() in orf.description
    assert orf.isorf()
    display(orf.express())
    hit, *rest_of_hits = genbank_accession(str(orf.seq))
    item, start, stop = hit
    genes.append(GenbankRecord(orf, item=item, start=start, stop=stop, strand=1))

# ATF1

cds,len,cai,gc,sta,stp,n-end,CGA,CGG,CGC,CCG,CTC,GCG,rare
ATG...TAG,526.0,0.738,0.377,1.0,0.23,3 min,1,4,3,5,6,1,0.038


# CTT1

cds,len,cai,gc,sta,stp,n-end,CGA,CGG,CGC,CCG,CTC,GCG,rare
ATG...TAA,563.0,0.767,0.413,1.0,0.47,3 min,0,0,0,3,1,1,0.009


In [56]:
genes

[Gbnk(-1578 CP072089 1046219-1047796), Gbnk(-1689 CP046087 678143-679831)]

In [23]:
#pathway_name = "pTA1_ENO2_CTT1_FBA1"
pathway_name = "pTA1_ENO2_CTT1_FBA1_ATF1_PGI1"

In [24]:
items = [n.strip().split("tp",1)[0] for n in pathway_name.split("_")]
items

['pTA1', 'ENO2', 'CTT1', 'FBA1', 'ATF1', 'PGI1']

In [25]:
backbone_vector, *elements = items

In [26]:
assert os.path.exists(sequence_library_path/f"{backbone_vector}.gb")

In [27]:
elements

['ENO2', 'CTT1', 'FBA1', 'ATF1', 'PGI1']

In [28]:
promoters, genes, terminators = elements[0:-1:2], elements[1::2], elements[2::2]

In [29]:
print("promoter(s)", promoters)
print("gene(s)", genes)
print("terminator(s)", terminators)

promoter(s) ['ENO2', 'FBA1']
gene(s) ['CTT1', 'ATF1']
terminator(s) ['FBA1', 'PGI1']


In [31]:
cassette_vector_paths = []
for p, g, t in zip(promoters, genes, terminators):
    cassette_vector_path = Path( f"{backbone_vector}_{p}_{g}_{t}.gb" )
    print(cassette_vector_path.name)
    cassette_vector_paths.append(cassette_vector_path)

pTA1_ENO2_CTT1_FBA1.gb
pTA1_FBA1_ATF1_PGI1.gb
